## String matching - lab 1

In [10]:
from time import time

#### Naive algorithm

In [6]:
def naive_match(text, pattern):
    n, m = len(text), len(pattern)
    result = []
    for s in range(0, n - m + 1):
        if (pattern == text[s:s + len(pattern)]):
            result.append(s)
            
    return result

#### Finite Automata

In [7]:
def fa_string_matching(text, pattern):
    delta = transition_table(pattern)
    q = 0
    length = len(delta) - 1
    result = []
    for s in range(0, len(text)):
        if text[s] in delta[q]:
            q = delta[q][text[s]]
            if(q == length):
                result.append(s + 1 - q)
        else:
            q = 0
    return result
    
        
def transition_table(pattern):
    M = len(pattern)
    alphabet = set(pattern)
    result = []
    for q in range(0, M + 1):
        result.append({})
        for a in alphabet:
            k = min(M, q + 1)
            suffix = pattern[:q] + a
            while k > 0 and pattern[:k] != suffix[q-k+1:]:
                k -= 1
            result[q][a] = k
    return result  

#### KMP

In [8]:
def prefix_function(pattern):
    pi = [0]
    k = 0
    for q in range(1, len(pattern)):
        while(k > 0 and pattern[k] != pattern[q]):
            k = pi[k-1]
        if(pattern[k] == pattern[q]):
            k = k + 1
        pi.append(k)
    return pi

def kmp_string_matching(text, pattern):
    result = []
    pi = prefix_function(pattern)
    q = 0
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = pi[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            result.append(i + 1 - q)
            q = pi[q-1]
    return result

#### Test with article

In [12]:
with open("test.txt", "r") as file:
    text = file.read()
    pattern = "Art"
    print("Check if all algorithms have same result")
    print(naive_match(text, pattern) == fa_string_matching(text, pattern) == kmp_string_matching(text, pattern))
    print("Lets measure how long did it take")
    f = [("Naive match", naive_match), ("Finite automation", fa_string_matching), ("KMP algorithm", kmp_string_matching)]
    for name, method in f:
        start = time()
        method(text, pattern)
        finish = time()
        print(f"{name} took {finish - start}s")

Check if all algorithms have same result
True
Lets measure how long did it take
Naive match took 0.03514528274536133s
Finite automation took 0.018488645553588867s
KMP algorithm took 0.028649091720581055s


##### The complexity of naive-algorithm is O(m(n-m+1)), thats why we should use a FA algorithm or KMP that hava matching times linear. We have to remember about pre-procesing.Depends on a pattern, the preprocessing time of these two algorithm would differ. KMP is much more elegant for me and easier to understand.


#### 4. The worst case of naive algorithm we get when text and pattern containts the same letter, for example "B"

In [22]:
# its just for ex3 (time without pre-processing)
def kmp_string_matching_time(text, pattern):
    result = []
    pi = prefix_function(pattern)
    start = time()
    q = 0
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = pi[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            result.append(i + 1 - q)
            q = pi[q-1]
    finish = time()
    return finish - start
def fa_string_matching_time(text, pattern):
    delta = transition_table(pattern)
    start = time()
    q = 0
    length = len(delta) - 1
    result = []
    for s in range(0, len(text)):
        if text[s] in delta[q]:
            q = delta[q][text[s]]
            if(q == length):
                result.append(s + 1 - q)
        else:
            q = 0
    finish = time()
    return finish - start
def naive_match_time(text, pattern):
    n, m = len(text), len(pattern)
    start = time()
    result = []
    for s in range(0, n - m + 1):
        if (pattern == text[s:s + len(pattern)]):
            result.append(s)
    finish = time()
    return finish - start


#### lets take pattern as a "B" * 100000 and text as a "B"* 1000000

In [42]:
pattern = "B" * 100000
text = "B"* 1000000
print(f"Naive match took {naive_match_time(text, pattern)}")
print(f"Finite automation took {fa_string_matching_time(text, pattern)}")
print(f"KMP algorithm took {kmp_string_matching_time(text, pattern)}")

Naive match took 5.183069944381714
Finite automation took 0.1924726963043213
KMP algorithm took 0.30113840103149414


#### 5. Pre-procesing of a transisiton table depends on number of different chars. Thats why if we want to find worst case for fa algorithm, pathern must contains as much different chars as posible. To prove that, lets take as a example pattern = "qwertyuiopasdfghjklzxcvbnm" * 10

In [44]:
pattern = "qwertyuiopasdfghjklzxcvbnm" * 100
start = time()
transition_table(pattern)
finish = time()
print(f"Pre-procesing of finite automate took {finish - start}")
start = time()
prefix_function(pattern)
finish = time()
print(f"Pre-procesing of kmp algorithm took {finish - start}")

Pre-procesing of finite automate took 22.03469753265381
Pre-procesing of finite automate took 0.0004942417144775391


#### What a difference!